# 1. K-NN classifier

**Question** How does the k-nn algorithm work?

**Question** What are the parameters we need to choose to classify with k-NN? How can we choose them

**Question** What are the advantages and disadvantages of this algorithm? For which kind of dataset it would be appropriate and for which datasets it wouldn't?

**Question** How can we deal with nominal values in distance measurement?


In further part of the laboratory, we will perform a preprocessing of the data and a classification of a set of biomedical voice measurements. Some of them has been recorded for people with Parkinson's desease.

More about the dataset: https://archive.ics.uci.edu/ml/datasets/parkinsons

First, we load the required packages:

In [ ]:
import pandas as pd
import sklearn.model_selection
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt

# 2. Data loading and analysis of the attributes
Let's start with the data preparation, 
#### 2.1. Load the dataset from file parkinsons.csv into data frame using library pandas (pd.read_csv). Write the body of the read_data function to return a data frame with attributes and a list with class labels. Classes are available in 'status' column. You should also remove column 'name' from the data (see function drop of dataFrame https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html).

In [ ]:
def read_data(path):
    #DONE replace the following line with our code
    data = pd.read_csv(path)
    data = data.drop(['name'], axis=1)
    Y = data['status']
    X = data.drop('status', axis=1)
    return X , Y

data_X, data_Y = read_data("parkinsons.data")

Let's see what we have loaded.

In [ ]:
data_X.head()

In [ ]:
data_Y

#### 1.2. Let's analyse the given data. 
* How many attributes are in given data?
* Are the attributes on the common scale?
* Are observations equally distributed for sick and healthy people?

Plot the histogram of the assigned class and analyse the distribution.


In [ ]:
plt.hist(data_Y)

Plot the histograms of the first 5 attributes. 

In [ ]:
plt.hist(data_X.iloc[:, :5])

# 3. Train and test set selection

#### We want to build our classifier and test it on another set of observations.

To split data into train and test sets use train_test_split method from sklearn.model_selection module (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Use 80% of cases in train set and 20% in test set. 
Use random_state = 5 just to be sure we all have the same rows in train and test sets :)

split_data should return a tuple containing: dataframe with train set attributes, list of labels for train data, dataframe with test set attributes and a list of labels for test data.

In [ ]:
def split_data(data_X, data_Y, test_percent = 20, random_state=5):
    return train_test_split(data_X, data_Y, test_size=test_percent/100, random_state=random_state)
    
(train_X, test_X, train_Y, test_Y) = split_data(data_X, data_Y)
print("rows in train set: ", train_X.shape[0])
print("rows in test set:", test_X.shape[0])

# 4. Data standarization/normalization
#### 4.1. Use k-nn algorithm to classify the obtained test set using k=3. What is the accuracy of the classification?

Use KNeighborsClassifier class from sklearn.neighbors module. Useful methods: fit and predict. Classification accuracy can be obtained with accuracy_score method from sklearn.metrics. Function get_classification_accuracy should return the accuracy of classification of given test set on model build with train set.

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
def get_classification_accuracy(train_data_X, train_data_Y, test_data_X, test_data_Y, k = 3):
    neigh = KNeighborsClassifier(k)
    neigh.fit(train_data_X, train_data_Y)

    accuracy = metrics.accuracy_score(test_data_Y, neigh.predict(test_data_X))
    return accuracy

get_classification_accuracy(train_X, train_Y, test_X, test_Y)

#### 4.2. Perform some normalization or standarization of attributes. Then repeat the classification. Do the classification accuracy change?

You can use sklearn.preprocessing.StandardScaler, sklearn.preprocessing.MinMaxScaler or sklearn.preprocessing.MaxAbsScaler and their fit_transform/transform methods.

Try other standarization methods to verify the standarization procedure influence the classification accuracy.
standarize_train_and_test should return 2 dataFrames - with normalized train and normalized test sets.

In [ ]:

def standarize_train_and_test(train_X, test_X, option = 1):
    if option == 1:
        scaler = preprocessing.StandardScaler()
    elif option == 2:
        scaler = preprocessing.MinMaxScaler()
    else:
        scaler = preprocessing.MaxAbsScaler()

    norm_train_X = scaler.fit_transform(train_X)
    norm_test_X = scaler.transform(test_X)
    return norm_train_X, norm_test_X


norm_train_X, norm_test_X = standarize_train_and_test(train_X, test_X, 1)
print(get_classification_accuracy(norm_train_X, train_Y, norm_test_X, test_Y))

norm_train_X, norm_test_X = standarize_train_and_test(train_X, test_X, 2)
print(get_classification_accuracy(norm_train_X, train_Y, norm_test_X, test_Y))

norm_train_X, norm_test_X = standarize_train_and_test(train_X, test_X, 3)
print(get_classification_accuracy(norm_train_X, train_Y, norm_test_X, test_Y))

# 5. Choosing k value
Using obtained in previous exercices train set with normalization/standarization use k-nn algoritm using k from 1 to 20. Use 5-fold cross-validation within the train set to obtain the classification accuracy. Plot the obtained accuracy of the classification. Which k value seems to be the best for the given dataset?

See https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html for more info about cross validation in sklearn.

In [ ]:

accuracy_scores = []
for k in range(1, 21):
    neigh = KNeighborsClassifier(n_neighbors=k)
    # todo PYTANIE: czy używać tutaj norm_train_X, czy przedłużyć to o _test_X, z racji że używamy cross-walidacji
    cv_results = sklearn.model_selection.cross_validate(neigh, norm_train_X, train_Y, cv=5, scoring='accuracy')
    accuracy_scores.append(cv_results['test_score'].mean())

# print(accuracy_scores)
plt.figure(figsize=(10, 6))
plt.plot(range(1, 21), accuracy_scores, marker='o', color='b', linestyle='-', markersize=5)
plt.xlabel("k")
plt.ylabel("Mean Cross-Validated Accuracy")
plt.title("k-NN Accuracy")
plt.grid()
plt.show()

best_k = accuracy_scores.index(max(accuracy_scores)) + 1
print(f"The best k value is {best_k} with a mean (training) accuracy of {max(accuracy_scores)}")


# 6. Testing classifier
Train the k-NN classifier again and test it using the obtained best k value on a test set to check the final classification accuracy. You can just call the previous written function get_classification_accuracy)

In [ ]:
final_accuracy = get_classification_accuracy(norm_train_X, train_Y, norm_test_X, test_Y, k=best_k)
# print(best_k, final_accuracy)
print(final_accuracy)

# bonus, sprawdzenie pozostałych czy "najlepsze" k jest rzeczywiście najlepsze
# acc2scores = []
# for k in range(1, 21):
#     final_accuracy = get_classification_accuracy(norm_train_X, train_Y, norm_test_X, test_Y, k=k)
#     print(f"{k}: final_accuracy: {final_accuracy}")
#     acc2scores.append(final_accuracy)
# new_best_k = acc2scores.index(max(acc2scores)) + 1
# print(f"The best k value is {new_best_k} with accuracy of {max(acc2scores)}")


# 7. Homework
You are given a dataset containing information about 1600 red wines (winequality.csv) containing 11 attributes and assignment to one of the three wine quality classess: "poor", "medium" and "good". Perform a preprocessing of this dataset (normalization, standardization). Divide this set into the train and test sets. Choose experimentally the best value of k (using cross-validation) and perform classification using the  k-nn algoritm with the chosen k value. Write a report containing information about the used preprocessing methods, chosen train/test split method (percentage of examples in train/test sets, if shuffling or stratification used), plot of the accuracy depending of k parameter, finally chosen k and obtained classification accuracy on a test set.

**Deadline +2 weeks**
    


In [ ]:
# use to standardize
def standardize_set(X, option = 1):
    if option == 1:
        scaler = preprocessing.StandardScaler()
    elif option == 2:
        scaler = preprocessing.MinMaxScaler()
    else:
        scaler = preprocessing.MaxAbsScaler()
    norm_X = scaler.fit_transform(X)
    return norm_X


In [ ]:
data = pd.read_csv("./winequality-red.csv")
# print(data)
Y = data['quality']
X = data.drop('quality', axis = 1)

# is it needed to split, if we use cv? anyway, sticking to plan
(train_X, test_X, train_Y, test_Y) = split_data(X, Y)
norm_train_X, norm_test_X = standarize_train_and_test(train_X, test_X, 1)

# norm_X = standardize_set(X, 1)
# print(norm_X)

# finding best k
accuracy_scores = []
for k in range(1, 21):
    neigh = KNeighborsClassifier(n_neighbors=k)
    cv_results = sklearn.model_selection.cross_validate(neigh, norm_train_X, train_Y, cv=10, scoring='accuracy')
    accuracy_scores.append(cv_results['test_score'].mean())

plt.figure(figsize=(10, 6))
plt.plot(range(1, 21), accuracy_scores, marker='o', color='b', linestyle='-', markersize=5)
plt.xlabel("k")
plt.ylabel("Mean Cross-Validated Accuracy")
plt.title("k-NN Accuracy")
plt.grid()
plt.show()
best_k = accuracy_scores.index(max(accuracy_scores)) + 1
print(f"The best k value is {best_k} with a mean (training) accuracy of {max(accuracy_scores)}")

# get final accuracy
final_accuracy = get_classification_accuracy(norm_train_X, train_Y, norm_test_X, test_Y, k=best_k)
print(f"The final accuracy is: {final_accuracy}")

# todo WRITE A REPORT


